In [ ]:
# -----------------------------------------------------------------------------------
# 파일명       : Fall_Detection.ipynb
# 설명         : 스켈레톤 데이터를 가지고 낙상 감지 모델 학습
# 작성자       : 이민하
# 작성일       : 2025-05-04
# 
# 사용 모듈    :
# - pandas                           # 데이터프레임 기반 데이터 처리
# - numpy                            # 수치 계산 및 배열 연산
# - os                               # 파일 및 경로 관리
# - sklearn.model_selection          # 학습/검증용 데이터 분할
# - torch, torch.nn, F               # PyTorch 모델 구축 및 연산
# - torch.optim, lr_scheduler        # 최적화 및 학습률 조정
# - torch.utils.data                 # 데이터셋 및 데이터로더 처리
# - torchmetrics.classification      # 분류 모델 평가 지표 계산

# -----------------------------------------------------------------------------------
# >> 주요 기능
# - 스켈레톤 데이터를 RNN 모델 종류를 통해 낙상인지 아닌지 시계열 분석
#
# >> 업데이트 내역
# [2025-05-04] 60fps 시퀀스 데이터 학습 (LSTM)
# [2025-05-06] 30fps 시퀀스 데이터 학습 (LSTM)
# [2025-05-10] 10fps 시퀀스 데이터 학습 (LSTM)
# [2025-05-12] GRU 모델로 변경, 데이터 증강
# [2025-05-22] 하이퍼파라미터 최종 튜닝
# -----------------------------------------------------------------------------------


In [ ]:
# 데이터프레임 기반 데이터 처리
import pandas as pd

# 수치 계산 및 배열 연산
import numpy as np

# 파일 및 경로 관리
import os

# 학습/검증용 데이터 분할
from sklearn.model_selection import train_test_split

# PyTorch 모델 구축 및 연산
import torch
import torch.nn as nn
import torch.nn.functional as F

# 최적화 및 학습률 조정
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

# 데이터셋 및 데이터로더 처리
from torch.utils.data import Dataset, DataLoader

# 분류 모델 평가 지표 계산
from torchmetrics.classification import BinaryF1Score

c:\Users\PNC\anaconda3\envs\Project_38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 데이터 및 라벨 경로 설정
DATA_PATH = "./10fps_merged_data(no interpolation).csv"
LABEL_PATH = "./Label.csv"

# 데이터 및 라벨 CSV 파일 -> DataFrame 변환
data_df = pd.read_csv(DATA_PATH)
label_df = pd.read_csv(LABEL_PATH)

In [ ]:
# 데이터 확인 
print(data_df.head())
print()
print()
print(label_df.head())

   frame  landmark_id         x         y         z  source_index
0    1.0          0.0  0.432988  0.559980 -0.354303             0
1    1.0         11.0  0.465422  0.526178 -0.257993             0
2    1.0         12.0  0.408942  0.558263 -0.288992             0
3    1.0         13.0  0.474997  0.550322 -0.326235             0
4    1.0         14.0  0.391634  0.598874 -0.350726             0


   source_index  label
0             0      0
1             1      0
2             2      0
3             3      0
4             4      0


In [ ]:
# 데이터 분포 확인
data_df["source_index"].value_counts()

source_index
2652    3003
2655    2899
2653    2782
2651    2366
2648    2314
        ... 
1464    1300
1465    1300
1466    1300
1467    1300
4271    1300
Name: count, Length: 3781, dtype: int64

In [ ]:
# 데이터 형태 맞추기
val = data_df["source_index"].value_counts().values
idx = data_df["source_index"].value_counts().index

# 13개의 관절을 10fps로 10초짜리 영상에서 추출하므로 13 * 10 * 10 = 1300
for i, num in enumerate(val):
    if num > 1300:
        data_df[data_df["source_index"] == idx[i]] = data_df[data_df["source_index"] == idx[i]][:1300]

data_df["source_index"].value_counts()

source_index
0.0       1300
2840.0    1300
2826.0    1300
2827.0    1300
2828.0    1300
          ... 
1451.0    1300
1452.0    1300
1453.0    1300
1454.0    1300
4271.0    1300
Name: count, Length: 3781, dtype: int64

In [ ]:
# 결측치 제거 (아예 추출이 안된 데이터 제거)
data_df.dropna(inplace = True)

In [ ]:
# 정렬
df = data_df.sort_values(by=["source_index", "frame"])

# 좌표(x, y, z) 별로 pivot
df_pivot = df.pivot(index=["frame", "source_index"], columns="landmark_id", values=["x", "y", "z"])

# 다중 인덱스 컬럼 -> 단일 문자열 변환 (예: ('x', 11) -> 'x_11')
df_pivot.columns = [f"{coord}_{int(lid)}" for coord, lid in df_pivot.columns]

# 인덱스 복구 및 정렬
df_pivot.reset_index(inplace=True)
df_pivot = df_pivot.sort_values(by=["source_index", "frame"]).reset_index(drop=True)

# 결과 CSV파일로 저장
df_pivot.to_csv("10fps_reshaped_data(no interpolation).csv", index=False)


In [ ]:
# 변형시킨 데이터 불러오기
data_df = pd.read_csv("10fps_reshaped_data(no interpolation).csv")

In [12]:
data_df["source_index"].value_counts()

source_index
0.0       100
2840.0    100
2826.0    100
2827.0    100
2828.0    100
         ... 
1451.0    100
1452.0    100
1453.0    100
1454.0    100
4271.0    100
Name: count, Length: 3781, dtype: int64

In [13]:
data_df.head()

,frame,source_index,x_0,x_11,x_12,x_13,x_14,x_15,x_16,x_23,...,z_13,z_14,z_15,z_16,z_23,z_24,z_25,z_26,z_29,z_30
0,1.0,0.0,0.432988,0.465422,0.408942,0.474997,0.391634,0.438641,0.427603,0.450574,...,-0.326235,-0.350726,-0.429115,-0.415761,0.009984,-0.010148,0.093614,0.096314,0.285863,0.288855
1,4.0,0.0,0.432988,0.465422,0.408942,0.474997,0.391634,0.438641,0.427603,0.450574,...,-0.326235,-0.350726,-0.429115,-0.415761,0.009984,-0.010148,0.093614,0.096314,0.285863,0.288855
2,7.0,0.0,0.432988,0.465422,0.408942,0.474997,0.391634,0.438641,0.427603,0.450574,...,-0.326235,-0.350726,-0.429115,-0.415761,0.009984,-0.010148,0.093614,0.096314,0.285863,0.288855
3,10.0,0.0,0.437716,0.465444,0.413786,0.473494,0.396775,0.440072,0.432454,0.449910,...,-0.306374,-0.344488,-0.399087,-0.399243,0.013423,-0.013590,0.110032,0.094801,0.306888,0.294059
4,13.0,0.0,0.442443,0.465467,0.418630,0.471991,0.401916,0.441504,0.437306,0.449246,...,-0.286514,-0.338250,-0.369060,-0.382725,0.016862,-0.017031,0.126450,0.093288,0.327914,0.299262


In [ ]:
# Feature 설정
X_list = []

for i in data_df["source_index"].unique():
    X_list.append(np.array(data_df[data_df["source_index"] == i].drop(["source_index"], axis = 1)))

X_list

In [ ]:
# Target 설정
y_list = list(label_df["label"])

y_list[-5:]

[1, 1, 1, 1, 1]

In [16]:
print(len(X_list), len(y_list))

3781 3781


In [ ]:
# Train, Test 데이터 나누기 (Test Size = 0.2)
X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, 
                                                    test_size = 0.2,
                                                    random_state = 42,
                                                    stratify = y_list)

In [18]:
len(X_train)

3024

In [ ]:
# 모델 연산을 위한 데이터셋
class DetectDataset(Dataset):
    def __init__(self, feature, target):
        self.feature = feature
        self.target = target
        self.n_rows = len(self.feature)

    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
        featureTS = torch.tensor(self.feature[index], dtype = torch.float32)
        targetTS = torch.tensor([self.target[index]], dtype = torch.float32)

        return featureTS, targetTS

In [ ]:
# Batch Size 설정
BATCH_SIZE = 64

# 데이터셋, 데이터로더 변환
trainDS = DetectDataset(X_train, y_train)
trainDL = DataLoader(trainDS, batch_size = BATCH_SIZE, shuffle = True)

In [ ]:
# 낙상 감지 모델
# 시퀀스 데이터에 강력한 RNN 계열 모델 사용
# LSTM 모델 -> GRU 모델 (라즈베리파이 경량화를 위해 변경)
class FallDetectModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, 
                 dropout, bidirectional):
        super().__init__()

        # GRU 모델
        self.model = nn.GRU(
            input_size = input_size,
            hidden_size = hidden_dim,
            num_layers = n_layers,
            dropout = dropout,
            bidirectional = bidirectional,
            batch_first = True
        )

        # 출력층
        # 양방향 LSTM (시퀀스 데이터에서 더 많은 정보 추출 가능)
        if bidirectional:
            self.output = nn.Linear(hidden_dim * 2 , 1)
        
        else:
            self.output = nn.Linear(hidden_dim, 1)
    
    def forward(self, inputs):
        output, _ = self.model(inputs)
        # 마지막 hidden state 선택
        output = output[:, -1, :]
        result = self.output(output)
        
        return result

In [ ]:
# 학습 파라미터 설정
EPOCH = 1000
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

# 모델 파라미터 설정
input_size = 40
hidden_dim = 64
n_layers = 2
dropout = 0.2
bidirectional = True

# 모델 생성
fall_detect_model = FallDetectModel(input_size = input_size, 
                                    hidden_dim = hidden_dim,
                                    n_layers = n_layers,
                                    dropout = dropout,
                                    bidirectional = bidirectional).to(DEVICE)

In [ ]:
# 모델 확인
fall_detect_model

FallDetectModel(
  (model): GRU(40, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (output): Linear(in_features=128, out_features=1, bias=True)
)

In [ ]:
# 손실, 평가 함수 생성
F1score = BinaryF1Score().to(DEVICE)
BCELoss = nn.BCEWithLogitsLoss()

# 옵티마이저 생성
optimizer = optim.Adam(fall_detect_model.parameters(), lr = LR)

# Learning Rate Scheduler 생성
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = "max", patience = 10, verbose = True)

In [ ]:
# 모델 Test 함수
def testing(model, feature, target):
    # Pytorch 학습을 위해 데이터프레임 -> 텐서 전환
    featureTS = torch.FloatTensor(feature).to(DEVICE)
    targetTS = torch.FloatTensor(target).to(DEVICE)

    # 데이터 형태 맞춤    
    targetTS = targetTS.unsqueeze(1)

    # Dropout, BatchNorm 등 가중치 규제 비활성화
    model.eval()

    # 평가를 위해 역전파 계산 X
    with torch.no_grad():
        pre_val = model(featureTS)
        loss_val = BCELoss(pre_val, targetTS)

        # 분류 문제이므로 sigmoid 활성화 함수 + 이진 분류 결과로 변경
        probs = torch.sigmoid(pre_val)
        preds = (probs > 0.5).int()

        score_val = F1score(preds, targetTS.int())
 

    return loss_val, score_val, preds

In [ ]:
# 모델 Train 함수
def training(model, trainDL, X_test, y_test):
    
    # 가중치 파일 저장 위치 정의
    SAVE_PATH = './saved_models/'
    os.makedirs(SAVE_PATH, exist_ok = True)

    # Early Stopping을 위한 변수
    BREAK_CNT_SCORE = 0
    LIMIT_VALUE = 200

    # Loss, Score 로그를 담을 리스트
    BCE_LOSS_HISTORY, SCORE_HISTORY = [[], []], [[], []]

    for epoch in range(1, EPOCH + 1):
        model.train()
        SAVE_WEIGHT = os.path.join(SAVE_PATH, f"model_weights_{epoch}.pth")

        bce_loss_total, score_total = 0, 0

        # Train DataLoader에 저장된 Feature, Target 텐서로 학습 진행
        for featureTS, targetTS in trainDL:
            # GPU 환경으로 데이터 이동
            featureTS = featureTS.to(DEVICE)
            targetTS = targetTS.to(DEVICE)
            
            # 결과 추론
            pre_val = model(featureTS)

            # 추론값으로 Loss값 계산
            bce_loss = BCELoss(pre_val, targetTS)
            
            # 활성화 함수 + 이진 분류 결과로 변경
            probs = torch.sigmoid(pre_val)
            preds = (probs > 0.5).int()

            # F1 Score 확인 (precision과 recall의 조화평균)
            score = F1score(preds, targetTS)

            bce_loss_total += bce_loss.item()
            score_total += score.item()

            # 이전 gradient 초기화
            optimizer.zero_grad()

            # 역전파로 gradient 계산
            bce_loss.backward()

            # 계산된 gradient로 가중치 업데이트
            optimizer.step()

        # Test Loss, Score, 예측값 계산
        test_bce_loss, test_score, test_preds = testing(model, X_test, y_test)
        
        BCE_LOSS_HISTORY[0].append(bce_loss_total / len(trainDL))
        SCORE_HISTORY[0].append(score_total / len(trainDL))

        BCE_LOSS_HISTORY[1].append(test_bce_loss)
        SCORE_HISTORY[1].append(test_score)

        print(f"[{epoch} / {EPOCH}]\n - TRAIN BCE LOSS : {BCE_LOSS_HISTORY[0][-1]}")
        print(f"- TRAIN F1 SCORE : {SCORE_HISTORY[0][-1]}")

        print(f"\n - TEST BCE LOSS : {BCE_LOSS_HISTORY[1][-1]}")
        print(f"- TEST F1 SCORE : {SCORE_HISTORY[1][-1]}")

        # Test Score 결과로 스케줄러 업데이트
        scheduler.step(test_score)

        # Early Stopping 구현
        if len(SCORE_HISTORY[1]) >= 2:
            if SCORE_HISTORY[1][-1] <= SCORE_HISTORY[1][-2]: BREAK_CNT_SCORE += 1

        if len(SCORE_HISTORY[1]) == 1:
            torch.save(model.state_dict(), SAVE_WEIGHT)
        
        else:
            if SCORE_HISTORY[1][-1] > max(SCORE_HISTORY[1][:-1]):
                torch.save(model.state_dict(), SAVE_WEIGHT)

        if BREAK_CNT_SCORE > LIMIT_VALUE:
            print(f"성능 및 손실 개선이 없어서 {epoch} EPOCH에 학습 중단")
            break

    return BCE_LOSS_HISTORY, SCORE_HISTORY

In [ ]:
# 모델 학습 시작
bce_loss, f1_score = training(fall_detect_model, trainDL, X_test, y_test)

[1 / 1000]
 - TRAIN BCE LOSS : 0.27001552501072484
- TRAIN F1 SCORE : 0.9020243858297666

 - TEST BCE LOSS : 0.35142141580581665
- TEST F1 SCORE : 0.8404558300971985
[2 / 1000]
 - TRAIN BCE LOSS : 0.2618290808362265
- TRAIN F1 SCORE : 0.9032264724373817

 - TEST BCE LOSS : 0.23300687968730927
- TEST F1 SCORE : 0.928909957408905
[3 / 1000]
 - TRAIN BCE LOSS : 0.2520702335362633
- TRAIN F1 SCORE : 0.9118649053076903

 - TEST BCE LOSS : 0.18664777278900146
- TEST F1 SCORE : 0.9399744868278503
[4 / 1000]
 - TRAIN BCE LOSS : 0.22905220851923028
- TRAIN F1 SCORE : 0.9212094731628895

 - TEST BCE LOSS : 0.19223730266094208
- TEST F1 SCORE : 0.9305912852287292
[5 / 1000]
 - TRAIN BCE LOSS : 0.21806890160466233
- TRAIN F1 SCORE : 0.926001970966657

 - TEST BCE LOSS : 0.17308580875396729
- TEST F1 SCORE : 0.9462102651596069
[6 / 1000]
 - TRAIN BCE LOSS : 0.2110835105801622
- TRAIN F1 SCORE : 0.9237878335018953

 - TEST BCE LOSS : 0.231827974319458
- TEST F1 SCORE : 0.9086021780967712
[7 / 1000]


In [29]:
for x, y in trainDL:
    print(x.shape)
    break

torch.Size([64, 100, 40])


In [30]:
video = r"./Test_Dataset/test_video2.mp4"

import cv2
import mediapipe as mp




In [31]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=0,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    smooth_landmarks=True
)

important_landmarks = [
    0, 11, 12, 13, 14, 15, 16,
    23, 24, 25, 26, 29, 30
]

test_video = pd.DataFrame(columns = ["frame", "landmark_id", "x", "y", "z"])

frame_count = 0
processed_frame = 0

cap = cv2.VideoCapture(video)
original_fps = cap.get(cv2.CAP_PROP_FPS)
skip_interval = max(1, round(original_fps / 10))

while cap.isOpened():
    ret, frame = cap.read()
    # 예: 90도 시계 방향으로 회전 (세로 영상이 눕는 경우)
    frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

    if not ret:
        break

    if frame_count % skip_interval == 0:
        # frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        processed_frame += 1
        frame = cv2.resize(frame, (640, 480))
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            for idx in important_landmarks:
                landmark = results.pose_landmarks.landmark[idx]
                test_video.loc[len(test_video)] = [processed_frame, idx, landmark.x, landmark.y, landmark.z]

    frame_count += 1

cap.release()


In [32]:
test_video

,frame,landmark_id,x,y,z
0,3.0,0.0,0.495077,0.202133,-0.259547
1,3.0,11.0,0.592108,0.307687,-0.000086
2,3.0,12.0,0.399682,0.322056,-0.001780
3,3.0,13.0,0.647337,0.425211,0.014860
4,3.0,14.0,0.329309,0.407353,0.015185
...,...,...,...,...,...
1477,116.0,24.0,0.371051,0.565896,-0.008246
1478,116.0,25.0,0.573948,0.641452,-0.297894
1479,116.0,26.0,0.377047,0.724311,-0.145936
1480,116.0,29.0,0.581554,0.861283,-0.186450


In [33]:
# 전체 프레임과 랜드마크 ID 목록 생성
all_frames = np.arange(1, 301, 3)
all_landmarks = test_video['landmark_id'].unique()

df_frames = test_video["frame"].unique()

frame_list = []
landmark_list = []

for frame in all_frames:
    if frame not in df_frames:
        for landmark in all_landmarks:
            frame_list.append(frame)
            landmark_list.append(landmark)

add_row = pd.DataFrame({"frame" : frame_list, "landmark_id" : landmark_list})

test_video = pd.concat([test_video, add_row], ignore_index = True)

test_video = test_video.groupby("landmark_id").apply(lambda x : x.sort_values(by = ["frame", "landmark_id"]).interpolate().ffill().bfill()).reset_index(drop=True)
test_video = test_video.sort_values(by = ["frame", "landmark_id"])

if len(test_video) > 3900:
    test_video = test_video[:3900]

test_video

,frame,landmark_id,x,y,z
0,1.0,0.0,0.495077,0.202133,-0.259547
176,1.0,11.0,0.592108,0.307687,-0.000086
352,1.0,12.0,0.399682,0.322056,-0.001780
528,1.0,13.0,0.647337,0.425211,0.014860
704,1.0,14.0,0.329309,0.407353,0.015185
...,...,...,...,...,...
1583,298.0,24.0,0.371051,0.565896,-0.008246
1759,298.0,25.0,0.573948,0.641452,-0.297894
1935,298.0,26.0,0.377047,0.724311,-0.145936
2111,298.0,29.0,0.581554,0.861283,-0.186450


In [34]:
# frame 오름차순, source_index 우선 정렬
test_video = test_video.sort_values(by=["frame"])

# 피벗: (frame, source_index) 기준, landmark_id 별 x/y/z를 칼럼으로
df_pivot = test_video.pivot(index=["frame"], columns="landmark_id", values=["x", "y", "z"])

# 다중 인덱스 컬럼을 단일 열로 변환: ex) ('x', 11.0) -> x_11
df_pivot.columns = [f"{coord}_{int(lid)}" for coord, lid in df_pivot.columns]

# 인덱스 복구
df_pivot.reset_index(inplace=True)
                     
# 다시 source_index 기준으로 묶고, 그 안에서 frame 오름차순 정렬 (보장용)
df_pivot = df_pivot.sort_values(by=["frame"]).reset_index(drop=True)

df_pivot
 

,frame,x_0,x_11,x_12,x_13,x_14,x_15,x_16,x_23,x_24,...,z_13,z_14,z_15,z_16,z_23,z_24,z_25,z_26,z_29,z_30
0,1.0,0.495077,0.592108,0.399682,0.647337,0.329309,0.651901,0.310710,0.537525,0.436034,...,0.014860,0.015185,-0.122630,-0.105123,0.022949,-0.022683,-0.287023,-0.434515,-0.242776,-0.385776
1,3.0,0.495077,0.592108,0.399682,0.647337,0.329309,0.651901,0.310710,0.537525,0.436034,...,0.014860,0.015185,-0.122630,-0.105123,0.022949,-0.022683,-0.287023,-0.434515,-0.242776,-0.385776
2,4.0,0.493193,0.595036,0.408497,0.678957,0.333286,0.661445,0.311194,0.553189,0.450343,...,-0.037487,-0.021422,-0.183644,-0.139331,0.013685,-0.013566,-0.136108,-0.406317,0.093499,-0.230269
3,5.0,0.492759,0.595715,0.412542,0.680569,0.338718,0.664453,0.312522,0.554868,0.456708,...,-0.043242,-0.012215,-0.169574,-0.093957,0.014683,-0.014548,-0.033808,-0.260285,0.177826,-0.076468
4,6.0,0.492311,0.596074,0.411140,0.677054,0.339470,0.664882,0.313429,0.554650,0.455327,...,0.087922,0.113832,-0.044071,0.013093,0.016334,-0.016176,-0.094358,-0.319006,0.029275,-0.234655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,286.0,0.342838,0.469944,0.267143,0.603506,0.223710,0.589624,0.249557,0.485274,0.371051,...,-0.009098,0.022492,-0.133869,-0.081422,0.007985,-0.008246,-0.297894,-0.145936,-0.186450,0.125205
172,289.0,0.342838,0.469944,0.267143,0.603506,0.223710,0.589624,0.249557,0.485274,0.371051,...,-0.009098,0.022492,-0.133869,-0.081422,0.007985,-0.008246,-0.297894,-0.145936,-0.186450,0.125205
173,292.0,0.342838,0.469944,0.267143,0.603506,0.223710,0.589624,0.249557,0.485274,0.371051,...,-0.009098,0.022492,-0.133869,-0.081422,0.007985,-0.008246,-0.297894,-0.145936,-0.186450,0.125205
174,295.0,0.342838,0.469944,0.267143,0.603506,0.223710,0.589624,0.249557,0.485274,0.371051,...,-0.009098,0.022492,-0.133869,-0.081422,0.007985,-0.008246,-0.297894,-0.145936,-0.186450,0.125205


In [35]:
model = fall_detect_model

# 2. 그런 다음 state_dict를 로드합니다
model.load_state_dict(torch.load('./Final_model/GRU(64, 97.6%, 100frames).pth'))

C:\Users\PNC\AppData\Local\Temp\ipykernel_23976\2943254839.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./Final_model/GRU(64, 97.6%,

<All keys matched successfully>

In [36]:
X_tensor = torch.tensor(df_pivot.values, dtype=torch.float32).to(DEVICE)
X_tensor = X_tensor.unsqueeze(0)

In [37]:
fall_detect_model.eval()
with torch.no_grad():
    output = fall_detect_model(X_tensor)  # 출력: (batch_size, 1)
    probs = torch.sigmoid(output)         # 확률화
    predicted_classes = (probs > 0.5).int()  # 0 또는 1로 변환
print(probs)
print("예측 결과:", predicted_classes)


tensor([[0.0056]], device='cuda:0')
예측 결과: tensor([[0]], device='cuda:0', dtype=torch.int32)


In [38]:
model.eval()
model = model.to('cpu')  # 디바이스 일치

example_input = torch.randn(1, 100, 40)
traced_model = torch.jit.trace(model, example_input)
traced_model.save("./Final_model/model_script(64).pt")


In [39]:
print(traced_model.graph)
print(traced_model.code)

graph(%self.1 : __torch__.FallDetectModel,
      %inputs : Float(1, 100, 40, strides=[4000, 40, 1], requires_grad=0, device=cpu)):
  %output : __torch__.torch.nn.modules.linear.Linear = prim::GetAttr[name="output"](%self.1)
  %model : __torch__.torch.nn.modules.rnn.GRU = prim::GetAttr[name="model"](%self.1)
  %109 : Tensor = prim::CallMethod[name="forward"](%model, %inputs)
  %71 : int = prim::Constant[value=0]() # C:\Users\PNC\AppData\Local\Temp\ipykernel_23976\20476242.py:23:0
  %72 : int = prim::Constant[value=0]() # C:\Users\PNC\AppData\Local\Temp\ipykernel_23976\20476242.py:23:0
  %73 : int = prim::Constant[value=9223372036854775807]() # C:\Users\PNC\AppData\Local\Temp\ipykernel_23976\20476242.py:23:0
  %74 : int = prim::Constant[value=1]() # C:\Users\PNC\AppData\Local\Temp\ipykernel_23976\20476242.py:23:0
  %75 : Float(1, 100, 128, strides=[128, 128, 1], requires_grad=1, device=cpu) = aten::slice(%109, %71, %72, %73, %74) # C:\Users\PNC\AppData\Local\Temp\ipykernel_23976\20476242

In [40]:
X_tensor = X_tensor.to('cpu')

torch.sigmoid(traced_model(X_tensor))

tensor([[0.0056]], grad_fn=<SigmoidBackward0>)